In [1]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [24]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2025-03-08 12:52:15--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 172.67.70.149, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip.2’

book-crossings.zip. 100%[===================>]  24.88M  97.9MB/s    in 0.3s    

2025-03-08 12:52:16 (97.9 MB/s) - ‘book-crossings.zip.2’ saved [26085508/26085508]

Archive:  book-crossings.zip
replace BX-Book-Ratings.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace BX-Books.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 
error:  invalid response [{ENTER}]
replace BX-Books.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [35]:
# Import CSV data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding="ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'}
)

df_ratings = pd.read_csv(
    ratings_filename,
    encoding="ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'}
)

# Preprocessing: Remove users with fewer than 200 ratings and books with fewer than 100 ratings
user_counts = df_ratings['user'].value_counts()
book_counts = df_ratings['isbn'].value_counts()

# Users with 200 or more ratings
valid_users = user_counts[user_counts >= 200].index
# Books with 100 or more ratings
valid_books = book_counts[book_counts >= 100].index

# Filter the DataFrames to keep only valid users and books
df_ratings_filtered = df_ratings[df_ratings['user'].isin(valid_users) & df_ratings['isbn'].isin(valid_books)]

# Merge the filtered DataFrames with books
merged_df = pd.merge(df_ratings_filtered, df_books, on="isbn")

# Create the user-book matrix
ratings_matrix = merged_df.pivot_table(index="user", columns="title", values="rating", fill_value=0)

# Initialize the KNN model
knn = NearestNeighbors(n_neighbors=6, metric='cosine', algorithm='brute')

# Train the model with the transposed matrix (books are columns)
knn.fit(ratings_matrix.T)


NearestNeighbors(algorithm='brute', metric='cosine', n_neighbors=6)

In [45]:
def get_recommends(book=""):
    if book not in ratings_matrix.columns:
        return [book, []]

    # Find the index of the book in the matrix
    book_index = ratings_matrix.columns.get_loc(book)

    # Get the nearest neighbors for the book
    distances, indices = knn.kneighbors(ratings_matrix.iloc[:, book_index].values.reshape(1, -1), n_neighbors=6)

    # Prepare the recommended books list
    recommended_books = []
    for i in range(1, 6):  # Skip the first one (itself)
        recommended_title = ratings_matrix.columns[indices.flatten()[i]]
        distance = distances.flatten()[i]
        recommended_books.append([recommended_title, distance])

    # Sort the recommendations by distance (descending order)
    recommended_books = sorted(recommended_books, key=lambda x: x[1], reverse=True)

    return [book, recommended_books]


In [46]:
book = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(book)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [["I'll Be Seeing You", 0.8016211], ['The Weight of Water', 0.77085835], ['The Surgeon', 0.7699411], ['I Know This Much Is True', 0.7677075], ['The Lovely Bones: A Novel', 0.7234864]]]
You passed the challenge! 🎉🎉🎉🎉🎉
